In [1]:
import cv2
from darkflow.net.build import TFNet
import numpy as np
import time

/home/admin238/anaconda3/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
W0723 13:05:03.431669 139921330394944 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

W0723 13:05:03.571215 139921330394944 deprecation_wrapper.py:118] From /home/admin238/Desktop/previous/ML_WEEK_11/darkflow-master/darkflow/net/build.py:15: The name tf.train.RMSPropOptimizer is deprecated. Please use tf.compat.v1.train.RMSPropOptimizer instead.



In [2]:
options = {
    'model': 'cfg/yolo.cfg',
    'load': 'bin/yolo.weights',
    'threshold': 0.2,
    #'gpu': 1.0
}


In [3]:
tfnet = TFNet(options)
colors = [tuple(255 * np.random.rand(3)) for _ in range(10)]

Parsing ./cfg/yolo.cfg
Parsing cfg/yolo.cfg
Loading bin/yolo.weights ...
Successfully identified 203934260 bytes
Finished in 0.13575148582458496s
Model has a coco model name, loading coco labels.

Building net ...


W0723 13:05:11.081860 139921330394944 deprecation_wrapper.py:118] From /home/admin238/Desktop/previous/ML_WEEK_11/darkflow-master/darkflow/net/build.py:105: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0723 13:05:11.322305 139921330394944 deprecation_wrapper.py:118] From /home/admin238/Desktop/previous/ML_WEEK_11/darkflow-master/darkflow/net/ops/simple.py:106: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.



Source | Train? | Layer description                | Output size
-------+--------+----------------------------------+---------------
       |        | input                            | (?, 608, 608, 3)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 608, 608, 32)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 304, 304, 32)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 304, 304, 64)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 152, 152, 64)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 152, 152, 128)
 Load  |  Yep!  | conv 1x1p0_1  +bnorm  leaky      | (?, 152, 152, 64)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 152, 152, 128)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 76, 76, 128)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 76, 76, 256)
 Load  |  Yep!  | conv 1x1p0_1  +bnorm  leaky      | (?, 76, 76, 128)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 76, 76, 256)
 Load  |  Yep!  | 

W0723 13:05:15.601858 139921330394944 deprecation.py:506] From /home/admin238/Desktop/previous/ML_WEEK_11/darkflow-master/darkflow/net/ops/convolution.py:28: calling extract_image_patches (from tensorflow.python.ops.array_ops) with ksizes is deprecated and will be removed in a future version.
Instructions for updating:
ksizes is deprecated, use sizes instead


 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 19, 19, 1024)
 Load  |  Yep!  | concat [16]                      | (?, 38, 38, 512)
 Load  |  Yep!  | conv 1x1p0_1  +bnorm  leaky      | (?, 38, 38, 64)
 Load  |  Yep!  | local flatten 2x2                | (?, 19, 19, 256)
 Load  |  Yep!  | concat [27, 24]                  | (?, 19, 19, 1280)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 19, 19, 1024)
 Load  |  Yep!  | conv 1x1p0_1    linear           | (?, 19, 19, 425)
-------+--------+----------------------------------+---------------
Running entirely on CPU
Finished in 9.676294326782227s



In [6]:
capture = cv2.VideoCapture(0)
capture.set(cv2.CAP_PROP_FRAME_WIDTH, 1720)
capture.set(cv2.CAP_PROP_FRAME_HEIGHT, 980)

while True:
    stime = time.time()
    ret, frame = capture.read()
    if ret:
        results = tfnet.return_predict(frame)
        for color, result in zip(colors, results):
            tl = (result['topleft']['x'], result['topleft']['y'])
            br = (result['bottomright']['x'], result['bottomright']['y'])
            label = result['label']
            confidence = result['confidence']
            text = '{}: {:.0f}%'.format(label, confidence * 100)
            frame = cv2.rectangle(frame, tl, br, color, 5)
            frame = cv2.putText(
                frame, text, tl, cv2.FONT_HERSHEY_COMPLEX, 1, (0, 0, 0), 2)
        cv2.imshow('frame', frame)
        print('FPS {:.1f}'.format(1 / (time.time() - stime)))
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

capture.release()
cv2.destroyAllWindows()


FPS 0.4
FPS 2.3
FPS 2.8
FPS 2.7
FPS 2.6
FPS 2.6
FPS 2.7
FPS 2.5
FPS 2.7
FPS 2.7
FPS 2.8
FPS 2.7
FPS 2.7
FPS 1.8
FPS 2.7
FPS 2.9
FPS 2.7
FPS 2.7
FPS 2.8
FPS 2.7
FPS 2.8
FPS 2.8
FPS 2.8
FPS 2.8
FPS 2.7
FPS 2.8
FPS 2.8
FPS 2.7
FPS 2.8
FPS 2.8
FPS 2.7
FPS 2.7
FPS 2.6
FPS 2.7
FPS 2.7
FPS 2.7
FPS 2.3
FPS 2.1
FPS 2.7
FPS 2.7
FPS 2.8
FPS 2.8
FPS 2.8
FPS 2.7
FPS 2.7
FPS 2.8
FPS 2.8
FPS 2.8
FPS 2.7
FPS 2.7
FPS 2.8
FPS 2.6
FPS 2.7
FPS 2.7
FPS 2.6
FPS 2.7
FPS 2.5
FPS 2.8
FPS 2.7
FPS 2.7
FPS 2.7
FPS 2.7
FPS 2.8
FPS 2.7
FPS 2.8
FPS 2.7
FPS 2.8
FPS 2.7
FPS 2.8
FPS 2.7
FPS 2.7
FPS 2.8
FPS 2.7
FPS 2.8
FPS 2.7
FPS 2.7
FPS 2.8
FPS 2.8
FPS 2.7
FPS 2.8
FPS 2.7
FPS 2.8
FPS 2.7
FPS 2.8
FPS 2.8
FPS 2.8
FPS 2.7
FPS 2.7
FPS 2.8
FPS 2.7
FPS 2.8
FPS 2.7
FPS 2.7
FPS 2.7
FPS 2.7
FPS 2.7
FPS 2.7
FPS 2.7
FPS 2.7
FPS 2.7
FPS 2.7
FPS 2.7
FPS 2.7
FPS 2.7
FPS 2.7
FPS 2.7
FPS 2.7
FPS 2.7
FPS 2.8
FPS 2.7
FPS 2.8
FPS 2.7
FPS 2.8
FPS 2.7
FPS 2.8
FPS 2.7
FPS 2.8
FPS 2.7
FPS 2.8
FPS 2.7
FPS 2.8
FPS 2.7
FPS 2.8
FPS 2.7
FPS 2.7
